In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind
import sys

n_pcs = range(5, 100, 5)
non_lin = [1]
non_lin_dict = {1: "sigmoid", 2: "relu"}
alpha = 0

#empty dictionar     

best_model = {k: (-np.inf, None) for k in data.keys()}
#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 150) 
        
        w, o, s  = model.get_params()
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%               100%
[                   ]

testing non-linearity:  sigmoid ... testing  5  pc's...
new best model for cell  517442753 . r2 test:  0.0421377752596
new best model for cell  517444738 . r2 test:  -0.00904535130945
new best model for cell  517443331 . r2 test:  -0.042551117025
new best model for cell  517442436 . r2 test:  0.0309714810225
new best model for cell  517445575 . r2 test:  0.00843991164949
new best model for cell  517444426 . r2 test:  0.00774223325208
new best model for cell  517442471 . r2 test:  0.0263058824232
new best model for cell  517443181 . r2 test:  0.0293795971458
new best model for cell  517443087 . r2 test:  -0.0310793730531
new best model for cell  517446833 . r2 test:  0.0152319492237
new best model for cell  517443827 . r2 test:  0.0148831945632
new best model for cell  517443924 . r2 test:  0.0125808198598
new best model for cell  517443957 . r2 test:  -0.0282516143187
new best model for cell  517442722 . r2 test:  -0.0381797310237
new best model for cell  517446488 . r2 test:  0.064538

[##                 ] | ETA: 00:10:45

new best model for cell  517444738 . r2 test:  0.0171584509546
new best model for cell  517443331 . r2 test:  0.0226385193466
new best model for cell  517443181 . r2 test:  0.036503051522
new best model for cell  517443087 . r2 test:  0.0172611388728
new best model for cell  517443957 . r2 test:  0.0341104301145
new best model for cell  517442722 . r2 test:  -0.0029578946334
new best model for cell  517442675 . r2 test:  0.0090209938865
new best model for cell  517446554 . r2 test:  -0.0136226963986
new best model for cell  517444175 . r2 test:  0.010994947811
new best model for cell  517443571 . r2 test:  -0.0201154321454
testing non-linearity:  sigmoid ... testing  15  pc's...


[###                ] | ETA: 00:16:49

...

[################   ] | ETA: 00:12:57

new best model for cell  517445575 . r2 test:  0.0610275141086
new best model for cell  517443924 . r2 test:  0.0300322919712
testing non-linearity:  sigmoid ... testing  90  pc's...


[################## ] | ETA: 00:06:57

testing non-linearity:  sigmoid ... testing  95  pc's...


[###################] | ETA: 00:00:00
Total time elapsed: 02:21:36
